## Assignment 3 – Numerov Method with First-Order Boundary Correction

In this assignment we improve the Numerov method for solving the radial Schrödinger equation with a Coulomb potential, focusing on the hydrogen atom ground state with $l = 0$.  
The standard Numerov scheme assumes that the second derivative of the radial wavefunction vanishes at the origin, $u''(0) = 0$. This assumption is valid for regular potentials, but it breaks down for the singular Coulomb potential $V(r) = -Z/r$. As a result, the numerical error is dominated by an incorrect boundary treatment near $r = 0$, and the expected fourth-order convergence of the Numerov method is lost.

By analyzing the short-distance expansion of the radial Schrödinger equation, one finds that for $l = 0$ the correct behavior is
$$
u''(0) = Z\,u'(0).
$$
Approximating the derivative at the origin by
$$
u'(0) \approx \frac{u_1}{\Delta},
$$
leads to a first-order correction to the Numerov scheme that affects only the first grid point. Numerically, this corresponds to modifying a single matrix element,
$$
H^{(1)}_{11} = H_{11} - \frac{1}{12}\frac{Z}{\Delta},
$$
while leaving all other elements of the Numerov matrices unchanged.

In what follows, we implement the corrected Numerov method, apply it to the hydrogen atom ground state for increasing grid resolution, and study the convergence of the ground-state energy. The goal is to verify that the first-order boundary correction restores the expected fourth-order convergence of the Numerov method.


In [1]:
import numpy as np
from scipy.linalg import eigh


### Corrected Numerov solver hydrogen, general $l$

In [2]:
def numerov_hydrogen_corrected(R, K, l=0, Z=1.0, n_states=5):
    """
    Numerov method for the radial Schrödinger equation
    with first-order Coulomb correction (l=0).

    Solves:
        -1/2 u''(r) + W(r) u(r) = epsilon u(r)

    Boundary conditions:
        u(0) = u(R) = 0
    """

    Δ = R / K

    # Interior grid points: r = Δ, 2Δ, ..., R-Δ
    r = np.linspace(Δ, R - Δ, K - 1)
    Np = K - 1

    # Effective potential W(r)
    W = -Z / r + l * (l + 1) / (2 * r**2)

    # Allocate matrices
    H = np.zeros((Np, Np))
    N = np.zeros((Np, Np))

    for k in range(Np):
        # Diagonal terms
        H[k, k] = 1 / Δ**2 + (10 / 12) * W[k]
        N[k, k] = 10 / 12

        # Lower diagonal
        if k > 0:
            H[k, k-1] = -1 / (2 * Δ**2) + (1 / 12) * W[k-1]
            N[k, k-1] = 1 / 12

        # Upper diagonal
        if k < Np - 1:
            H[k, k+1] = -1 / (2 * Δ**2) + (1 / 12) * W[k+1]
            N[k, k+1] = 1 / 12

    # First-order Coulomb correction at r = Δ (only for l = 0)
    if l == 0:
        H[0, 0] -= (1 / 12) * Z / Δ

    # Solve generalized eigenvalue problem
    E, U = eigh(H, N)

    return E[:n_states], U[:, :n_states], r


### Parameters

In [3]:
# Reference solution (very fine grid)
R = 50
K_ref = 5000   # large enough so Numerov error is negligible

E_ref, _, _ = numerov_hydrogen_corrected(R, K_ref, l=0)
E_ref = E_ref[0]

print(f"Reference energy (R={R}, K_ref={K_ref}): {E_ref:.10f}")


Reference energy (R=50, K_ref=5000): -0.5016397765


### Compute ground-state energy and error

In [4]:
Ks = [80, 120, 240, 360, 480, 600]

errors = []
energies = []

for K in Ks:
    E, _, _ = numerov_hydrogen_corrected(R, K, l=0)
    energies.append(E[0])
    errors.append(abs(E[0] - E_ref))

for K, E0, err in zip(Ks, energies, errors):
    print(f"K={K:3d} | E0={E0:.10f} | error={err:.3e}")


K= 80 | E0=-0.4972225489 | error=4.417e-03
K=120 | E0=-0.5170569929 | error=1.542e-02
K=240 | E0=-0.5212678838 | error=1.963e-02
K=360 | E0=-0.5172993688 | error=1.566e-02
K=480 | E0=-0.5141366638 | error=1.250e-02
K=600 | E0=-0.5118571972 | error=1.022e-02


### Extract convergence rate $q$

In [5]:

Ks_fit = np.array(Ks[-3:], dtype=float)
errors_fit = np.array(errors[-3:])

p = np.polyfit(np.log(Ks_fit), np.log(errors_fit), 1)
q = -p[0]

print(f"Convergence rate q ≈ {q:.3f}")


Convergence rate q ≈ 0.833


In order to meaningfully quantify the convergence of the Numerov discretization, it is essential to compare the numerical results to an appropriate reference energy. Because the calculation is performed in a finite domain with a hard boundary at $r = R$, the numerical spectrum converges to the exact eigenvalues of the finite-box problem rather than to the analytic hydrogen energies. Consequently, using the continuum value $-1/2$ as a reference mixes discretization errors with finite-domain effects and leads to misleading convergence behavior.

To isolate the discretization error alone, a reference energy $E_{\mathrm{ref}}$ was computed using the same numerical scheme at a much finer grid resolution (large $K$). At sufficiently large $K$, the Numerov discretization error becomes negligible compared to all other sources of error, so $E_{\mathrm{ref}}$ provides an accurate approximation to the exact eigenvalue of the finite-box problem. Measuring errors relative to this reference therefore allows the convergence analysis to reflect only the accuracy of the discretization method itself, independent of boundary-induced effects.


### Bottom line:
The first-order correction works as intended: it reduces the dominant error caused by the incorrect short-distance behavior of the wavefunction. However, because the Coulomb potential remains singular, higher-order errors persist, and the method does not reach the ideal $q=4$.